In [21]:
import pandas as pd
import matplotlib.pyplot as plt

def calculate_yield(yield_interest = 10, starting_fyt_interest = 5,
                    ending_fyt_interest = 10, step_size = .15,
                    input_amount = 100, liquidity = 4000, gas_fee = .0019,
                    days_maturity = 90, days_matured = 0,
                    time_stretch = 364 * 10):
  columns = (
    'X_Reserves',
    'PT Interest',
    'Interest W/ Slippage',
    'Total Expenditure',
    'Received at Maturity',
    'APY'
  )

  values = []
  current_interest = starting_fyt_interest

  # zero out accumulated interest
  accumulated_interest = 0

  # Include time stretch for *4
  t = (days_maturity - days_matured) / time_stretch

  for i in range (0, int(((ending_fyt_interest - starting_fyt_interest) / step_size)) + 1):
    if days_matured > 0:
      accumulated_interest = input_amount * days_matured/365 * yield_interest/100

    # If the apy is 10% and there is 6 months remaining, the applied interest is 5%
    applied_interest = (days_maturity - days_matured) / 365 * current_interest

    y_start = liquidity
    x_start = y_start/((1-applied_interest/100)**(-1/t)-1)
    total_supply = x_start

    k = (y_start + total_supply)**(1-t) + x_start**(1-t)
    j = input_amount - accumulated_interest
    i = x_start - pow(k-pow(y_start + total_supply + j,1-t),1/(1-t))
    fee = (j - i) * .1
    i = i - fee

    fyt_discount_with_slippage = (1 - (i / j)) * 100

    row = []
    row.append(x_start)
    row.append(current_interest)
    row.append(round(fyt_discount_with_slippage * (365/ (days_maturity - days_matured)), 2))
    fyt_expenditure = input_amount - accumulated_interest - i
    expenditure = fyt_expenditure + gas_fee + accumulated_interest

    row.append(expenditure)

    received = (input_amount * yield_interest / 100) * (days_maturity) / 365
    row.append(received)
    row.append(round(((received / expenditure) - 1) * 365 / (days_maturity - days_matured) * 100, 2))
    values.append(row)
    current_interest = current_interest + step_size

  df = pd.DataFrame(values, columns = columns)
  pd.options.display.width = 20
  df.style.set_properties(**{'text-align': 'left'})
  return df

In [24]:
import ipywidgets as widgets
from ipywidgets import interact


def foo(yield_interest,starting_fyt_interest):
    return calculate_yield(yield_interest,starting_fyt_interest);


print(interact(foo,yield_interest=(1,50,1),starting_fyt_interest=(1,10,1)))



interactive(children=(IntSlider(value=25, description='yield_interest', max=50, min=1), IntSlider(value=5, des…

<function foo at 0x7f8b4dd4dd30>
